In [1]:
import hl7
from datetime import datetime
from app.models import ErrorCharges,Patient
from asgiref.sync import sync_to_async

file_paths = ['app/HL7/charges_0.hl7', 'app/HL7/charges_1.hl7', 'app/HL7/charges_2.hl7', 'app/HL7/charges_3.hl7', 'app/HL7/charges_4.hl7']
def parse_hl7(file_path):
    with open(file_path, 'br') as file:
        data = file.read()
    return hl7.parse(data)

parsed_messages = [parse_hl7(file_path) for file_path in file_paths]
# def extract_patient_info(message):
#     patient_info = {}
#     segments = [segment for segment in message]
#     for segment in segments:
#         print(segment,'0000000000000000000000')
#         if segment[0] == 'PID':
#             patient_info['first_name'] = segment[5][0][1]
#             patient_info['last_name'] = segment[5][0][0]
#             patient_info['date_of_birth'] = segment[7]
#             patient_info['mrn'] = segment[3][0]
#         if segment[0] == 'PV1':
#             patient_info['physician_name'] = segment[7][1]
#             patient_info['phone'] = segment[13]
#     return patient_info

# patient_data = [extract_patient_info(message) for message in parsed_messages]
# print(patient_data)

# def extract_patient_info(message):
#     patient_info = {}
#     segments = [segment for segment in message]
#     for segment1 in segments:
#         segment = [segment2 for segment2 in segment1]
#         print(segment)
#         # if segment[0] == 'PID':
#         if segment[0] == ['\nPID']:
#             # print('10100101')
#             patient_info['first_name'] = segment[5][0][1][0]
#             patient_info['last_name'] = segment[5][0][0][0]
#             patient_info['date_of_birth'] = segment[7][0]
#             patient_info['mrn'] = segment[3][0][0]
#         if segment[0] == 'PV1':
#             patient_info['physician_name'] = segment[7][1]
#             patient_info['phone'] = segment[13]
#         # print(patient_info,'11111111111111111')
#     return patient_info

# patient_data = [extract_patient_info(message) for message in parsed_messages]
# print(patient_data)
patient_data = []
def extract_patient_info(message):
    segments = [segment for segment in message]
    for segment1 in segments:
        patient_info = {}
        segment = [segment2 for segment2 in segment1]
        # print(segment[0])
        # print(str(segment[5][0]).split('^^')[0].split('^')[0])
        # print(segment[7][0][0:4]+'-'+segment[7][0][4:6]+'-'+segment[7][0][6:8]  if segment[7][0] != '' else None,'---')
        # print(datetime.strptime(str(segment[7][0]), '%Y%m%d').date())
        # print(str(segment[11][0]).replace('^^','-').replace('^',', '),'--')
        # print(len(segment),'-')
        # if segment[0] == 'PID':
        if segment[0] == ['\nPID']:
            # print('10100101')
            fullnamelist = str(segment[5][0]).split('^^')
            namelist = fullnamelist[0].split('^')
            # print(' '.join(namelist[1:]) if ' '.join(namelist[1:]) != '' else None)
            # print(list(segment[3][0]))
            # print(list(segment[3][0])[0][0] if list(segment[3][0])[0] != [''] else None)
            patient_info['full_name'] = fullnamelist[0].replace('^',' ')  if fullnamelist[0].replace('^',' ') != '' else None
            patient_info['first_name'] = namelist[0] if namelist[0] != '' else None
            patient_info['last_name'] = ' '.join(namelist[1:]) if ' '.join(namelist[1:]) != '' else None
            patient_info['gender'] = 'Male' if segment[8][0] == 'M' else 'Female' if segment[8][0] == 'F' else None
            patient_info['phone'] = segment[13][0]
            patient_info['address'] = str(segment[11][0]).replace('^^','-').replace('^',', ')
            patient_info['date_of_birth'] = segment[7][0][0:4]+'-'+segment[7][0][4:6]+'-'+segment[7][0][6:8]  if segment[7][0] != '' else None
            patient_info['mrn'] = list(segment[3][0])[0][0] if list(segment[3][0])[0] != [''] else None
            patient_data.append(patient_info)
        # if segment[0] == ['\nPV1']:
        #     print(list(segment)[17])
            # patient_info['physician_name'] = segment[7][1]
            # patient_info['phone'] = segment[13]
        # print(patient_info,'11111111111111111')
    return patient_info



for message in parsed_messages:
    extract_patient_info(message)

# def store_patient_data(patient_data):
#     for data in patient_data:
#         print(data)
#         error_charges_id = None
#         if data['full_name'] == None:
#             error_charges_id = 1
#         if data['date_of_birth'] == None:
#             error_charges_id = 2
#         if data['mrn'] == None:
#             error_charges_id = 3
#         if error_charges_id:
#             data["error_charges_id"] = error_charges_id
#         try:
#             Patient.objects.create(**data)
#             # Patient.objects.create(
#             #     full_name=data['full_name'],
#             #     first_name=data['first_name'],
#             #     last_name=data['last_name'],
#             #     gender=data['gender'],
#             #     phone=data['phone'],
#             #     address=data['address'],
#             #     date_of_birth=data['date_of_birth'],
#             #     mrn=data['mrn'],
#             # )
#         except Exception as e:
#             print(e)
# store_patient_data(patient_data)

async def store_patient_data_async(patient_data):
    for data in patient_data:
        error_charges_id = None
        if data['full_name'] is None:
            error_charges_id = 1
        if data['date_of_birth'] is None:
            error_charges_id = 2
        if data['mrn'] is None:
            error_charges_id = 3
        if error_charges_id:
            data["error_charges_id"] = error_charges_id

        # Use sync_to_async to wrap the create operation
        await sync_to_async(Patient.objects.create)(**data)

# Example of calling the async function
await store_patient_data_async(patient_data)
